In [ ]:
# 1. INSTALACIÓN Y CONFIGURACIÓN INICIAL
# ======================================
# Instalar bibliotecas necesarias
!pip install streamlit joblib pandas numpy plotly scikit-learn lightgbm xgboost

# Instalar ngrok y pyngrok para visualizar Streamlit en Colab
!pip install pyngrok

# Importar bibliotecas
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import joblib
import sys
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import warnings
from pyngrok import ngrok
import subprocess
import time

warnings.filterwarnings('ignore')

# 2. GESTIÓN DE RUTAS Y ESTRUCTURA DE CARPETAS PARA COLAB
# ========================================================
# Definir la ruta raíz del proyecto para Colab
# En Colab, el script se ejecuta en el directorio actual, así que lo usamos como raíz.
def get_project_root():
    """Obtener la ruta raíz del proyecto: el directorio actual en Colab."""
    return os.getcwd()

PROJECT_ROOT = get_project_root()

# Crear la estructura de directorios necesaria
os.makedirs(os.path.join(PROJECT_ROOT, 'data', 'raw'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, 'models'), exist_ok=True)

# NOTA: Para que el entrenamiento funcione, debes subir tus archivos:
# kepler.csv, k2.csv y tess.csv
# a la carpeta 'data/raw' que acabamos de crear en el entorno de Colab.
# Puedes hacerlo manualmente usando el explorador de archivos de Colab (icono de carpeta).
print(f"Carpeta de datos creada en: {os.path.join(PROJECT_ROOT, 'data', 'raw')}")
print("¡Recuerda subir tus CSV de la NASA a esta carpeta!")

# 3. DEFINICIÓN DE CLASES Y FUNCIONES (Tu código original)
# =========================================================

class ExoplanetDataProcessor:
    """Procesador de datos para los datasets reales de la NASA"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.feature_names = []

    def load_real_data(self):
        """Cargar los datasets reales de la NASA con rutas corregidas"""
        try:
            # Usar rutas absolutas desde la raíz del proyecto
            data_dir = os.path.join(PROJECT_ROOT, 'data', 'raw')

            st.info(f"🔍 Buscando datos en: {data_dir}")

            # Listar archivos en el directorio
            if os.path.exists(data_dir):
                files = os.listdir(data_dir)
                st.info(f"📁 Archivos encontrados en data/raw/: {files}")
            else:
                st.error(f"❌ No existe el directorio: {data_dir}")
                return None, None, None

            # Construir rutas completas
            kepler_path = os.path.join(data_dir, 'kepler.csv')
            k2_path = os.path.join(data_dir, 'k2.csv')
            tess_path = os.path.join(data_dir, 'tess.csv')

            st.info(f"📊 Intentando cargar:\n- {kepler_path}\n- {k2_path}\n- {tess_path}")

            # Verificar que los archivos existen
            if not os.path.exists(kepler_path):
                st.error(f"❌ No existe: {kepler_path}")
                # Buscar archivos similares
                csv_files = [f for f in files if f.endswith('.csv')]
                if csv_files:
                    st.info(f"📄 Archivos CSV disponibles: {csv_files}")
                return None, None, None

            # Cargar los archivos reales
            kepler_df = pd.read_csv(kepler_path)
            k2_df = pd.read_csv(k2_path) if os.path.exists(k2_path) else None
            tess_df = pd.read_csv(tess_path) if os.path.exists(tess_path) else None

            st.success(f"✅ Kepler cargado: {len(kepler_df)} registros")
            if k2_df is not None:
                st.success(f"✅ K2 cargado: {len(k2_df)} registros")
            if tess_df is not None:
                st.success(f"✅ TESS cargado: {len(tess_df)} registros")

            return kepler_df, k2_df, tess_df

        except Exception as e:
            st.error(f"❌ Error cargando datasets: {e}")
            return None, None, None

    def preprocess_kepler(self, df):
        """Preprocesar datos Kepler reales - VERSIÓN MEJORADA"""
        df_clean = df.copy()

        st.info("🔧 Procesando datos Kepler...")

        # Mostrar columnas disponibles
        st.write(f"📋 Columnas en Kepler: {list(df_clean.columns)}")

        # Verificar si existe la columna de target
        if 'koi_disposition' not in df_clean.columns:
            st.error("❌ No se encuentra la columna 'koi_disposition' en Kepler")
            st.info("Las columnas disponibles son:")
            st.write(list(df_clean.columns))
            return df_clean

        # Eliminar columnas no útiles (basado en el paper)
        columns_to_drop = ['kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score']
        columns_to_drop = [col for col in columns_to_drop if col in df_clean.columns]

        if columns_to_drop:
            df_clean = df_clean.drop(columns=columns_to_drop)
            st.write(f"🗑️ Columnas eliminadas: {columns_to_drop}")

        # Mostrar valores únicos en la columna de disposición
        st.write(f"🎯 Valores en koi_disposition: {df_clean['koi_disposition'].unique()}")

        # Filtrar solo confirmed, candidate y false positive
        valid_dispositions = ['CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']
        mask = df_clean['koi_disposition'].isin(valid_dispositions)
        df_clean = df_clean[mask]

        st.write(f"📊 Distribución después de filtrar: {df_clean['koi_disposition'].value_counts().to_dict()}")

        # Crear target binario
        df_clean['target'] = df_clean['koi_disposition'].map({
            'CONFIRMED': 1,
            'CANDIDATE': 1,
            'FALSE POSITIVE': 0
        })

        # Añadir identificador de misión
        df_clean['mission'] = 'kepler'

        st.success(f"✅ Kepler procesado: {len(df_clean)} registros")

        return df_clean

    def preprocess_k2(self, df):
        """Preprocesar datos K2 reales"""
        if df is None:
            st.warning("⚠️ Dataset K2 no disponible")
            return None

        df_clean = df.copy()

        st.info("🔧 Procesando datos K2...")
        st.write(f"📋 Columnas en K2: {list(df_clean.columns)}")

        # Verificar columnas necesarias
        if 'disposition' not in df_clean.columns:
            st.error("❌ No se encuentra la columna 'disposition' en K2")
            return None

        # Filtrar solo confirmed y candidate
        df_clean = df_clean[df_clean['disposition'].isin(['CONFIRMED', 'CANDIDATE'])]

        # Target binario
        df_clean['target'] = df_clean['disposition'].map({
            'CONFIRMED': 1,
            'CANDIDATE': 1
        })

        # Identificador de misión
        df_clean['mission'] = 'k2'

        st.success(f"✅ K2 procesado: {len(df_clean)} registros")

        return df_clean

    def preprocess_tess(self, df):
        """Preprocesar datos TESS reales"""
        if df is None:
            st.warning("⚠️ Dataset TESS no disponible")
            return None

        df_clean = df.copy()

        st.info("🔧 Procesando datos TESS...")
        st.write(f"📋 Columnas en TESS: {list(df_clean.columns)}")

        # Verificar columnas necesarias
        if 'tfopwg_disp' not in df_clean.columns:
            st.error("❌ No se encuentra la columna 'tfopwg_disp' en TESS")
            return None

        # Mapear disposiciones de TESS
        disposition_mapping = {
            'PC': 1, 'KP': 1, 'APC': 1,  # Positivos
            'FP': 0, 'FA': 0  # Negativos
        }

        df_clean['target'] = df_clean['tfopwg_disp'].map(disposition_mapping)
        df_clean = df_clean.dropna(subset=['target'])

        # Identificador de misión
        df_clean['mission'] = 'tess'

        st.success(f"✅ TESS procesado: {len(df_clean)} registros")

        return df_clean

    def prepare_features(self, df):
        """Preparar características para el modelo - VERSIÓN FLEXIBLE"""
        if df is None or len(df) == 0:
            st.error("❌ No hay datos para preparar características")
            return None, None, None

        st.info("🔧 Preparando características...")

        # Posibles nombres de columnas en diferentes datasets
        possible_features = {
            'orbital_period': ['koi_period', 'pl_orbper', 'period'],
            'transit_duration': ['koi_duration', 'pl_trandurh', 'duration'],
            'transit_depth': ['koi_depth', 'pl_trandep', 'depth'],
            'planet_radius': ['koi_prad', 'pl_rade', 'radius'],
            'equilibrium_temp': ['koi_teq', 'pl_eqt', 'teq'],
            'insolation_flux': ['koi_insol', 'pl_insol', 'insol'],
            'stellar_teff': ['koi_steff', 'st_teff', 'teff'],
            'stellar_logg': ['koi_slogg', 'st_logg', 'logg'],
            'stellar_radius': ['koi_srad', 'st_rad', 'srad']
        }

        # Encontrar las columnas disponibles
        available_columns = []
        for feature_name, possible_names in possible_features.items():
            for name in possible_names:
                if name in df.columns:
                    available_columns.append(name)
                    break

        st.write(f"📊 Columnas numéricas encontradas: {available_columns}")

        if not available_columns:
            st.error("❌ No se encontraron columnas numéricas para entrenar")
            return None, None, None

        X = df[available_columns].copy()
        y = df['target'].values

        st.write(f"📊 Shape de X: {X.shape}, Shape de y: {y.shape}")

        # Manejar valores missing
        missing_before = X.isnull().sum().sum()
        X = X.fillna(X.median())
        missing_after = X.isnull().sum().sum()

        st.write(f"🔧 Valores missing: {missing_before} antes, {missing_after} después")

        # Escalar características
        # La forma correcta de escalar para entrenar es:
        # 1. Ajustar el escalador (fit_transform) si estás en la etapa de entrenamiento.
        # 2. Transformar (transform) si estás en la etapa de predicción.
        # Aquí es entrenamiento:
        X_scaled = self.scaler.fit_transform(X)
        self.feature_names = available_columns

        st.success(f"✅ Características preparadas: {X_scaled.shape}")

        return X_scaled, y, available_columns

class RealExoplanetModel:
    """Modelo real para entrenamiento con datos de la NASA"""

    def __init__(self):
        self.model = None
        self.accuracy = 0
        self.feature_importance = None

    def create_ensemble(self):
        """Crear ensemble con los algoritmos del paper"""
        base_models = [
            ('random_forest', RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                min_samples_split=5,
                random_state=42,
                n_jobs=-1
            )),
            ('extra_trees', ExtraTreesClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )),
            ('xgboost', XGBClassifier(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=6,
                random_state=42
            )),
            ('lightgbm', LGBMClassifier(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=6,
                random_state=42
            ))
        ]

        ensemble = StackingClassifier(
            estimators=base_models,
            final_estimator=LogisticRegression(),
            cv=3,  # Reducido para mayor velocidad
            passthrough=False,
            n_jobs=-1
        )

        return ensemble

    def train(self, X, y):
        """Entrenar el modelo real"""
        if X is None or y is None:
            st.error("❌ No hay datos para entrenar")
            return None

        st.info("🤖 Iniciando entrenamiento del ensemble...")

        self.model = self.create_ensemble()
        self.model.fit(X, y)

        # Calcular accuracy en entrenamiento
        y_pred = self.model.predict(X)
        self.accuracy = accuracy_score(y, y_pred)

        st.write(f"📈 Accuracy en entrenamiento: {self.accuracy:.2%}")

        # Calcular importancia de características
        self._calculate_feature_importance(X.shape[1])

        return self.model

    def _calculate_feature_importance(self, n_features):
        """Calcular importancia de características promediada"""
        importances = np.zeros(n_features)

        for name, model in self.model.named_estimators_.items():
            if hasattr(model, 'feature_importances_'):
                # Los modelos de árbol (RF, ET, XGB, LGBM) tienen feature_importances_
                importances += model.feature_importances_
            # Nota: StackingClassifier y LogisticRegression (final_estimator) no tienen
            # este atributo, por lo que solo promediamos sobre los que sí lo tienen.

        estimators_with_importance = sum(1 for name, model in self.model.named_estimators_.items() if hasattr(model, 'feature_importances_'))

        if estimators_with_importance > 0:
            self.feature_importance = importances / estimators_with_importance
        else:
            self.feature_importance = None # No se pudo calcular

    def save_model(self, filepath):
        """Guardar modelo entrenado"""
        if self.model:
            # Asegurar que el directorio existe
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            joblib.dump(self.model, filepath)
            return True
        return False

    def load_model(self, filepath):
        """Cargar modelo entrenado"""
        try:
            if os.path.exists(filepath):
                self.model = joblib.load(filepath)
                # Intenta cargar el accuracy si existe un archivo de métricas,
                # pero para simplificar lo dejamos en 0.
                self.accuracy = 0
                return True
        except Exception as e:
            st.error(f"Error cargando modelo: {e}")
        return False

class ExoplanetDetectorApp:
    def __init__(self):
        self.model = RealExoplanetModel()
        self.data_processor = ExoplanetDataProcessor()
        self.model_trained = False

        # Autocargar modelo y preprocesador al inicio si existen
        self.load_initial_state()

    def load_initial_state(self):
        """Cargar modelo y preprocesador guardados si existen"""
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        features_path = os.path.join(PROJECT_ROOT, 'models', 'feature_names.pkl')

        if os.path.exists(model_path):
            if self.model.load_model(model_path):
                self.model_trained = True

        if os.path.exists(processor_path):
            try:
                self.data_processor = joblib.load(processor_path)
            except Exception as e:
                # Si falla la carga del procesador, se ignora y se usa el nuevo.
                pass

        if os.path.exists(features_path):
            try:
                self.data_processor.feature_names = joblib.load(features_path)
            except Exception as e:
                # Si falla la carga de nombres, se ignora.
                pass


    def render_sidebar(self):
        """Barra lateral de navegación - ACTUALIZADA"""
        st.sidebar.title("🔭 NASA Exoplanet Detector - REAL")
        st.sidebar.markdown("---")

        page = st.sidebar.radio("Navegación", [
            "🏠 Inicio",
            "🚀 Entrenar Modelo REAL",
            "🤖 Clasificar Exoplanetas",
            "📦 Clasificación por Lotes",
            "📊 Análisis de Datos REAL",
            "💾 Modelos Guardados"
        ])

        st.sidebar.markdown("---")
        st.sidebar.info(
            "Sistema REAL con datos de Kepler, K2 y TESS de la NASA"
        )

        return page

    def render_home(self):
        """Página de inicio"""
        st.title("🪐 NASA Exoplanet Detection AI - SISTEMA REAL")

        col1, col2 = st.columns([2, 1])

        with col1:
            st.markdown("""
            ### Sistema REAL de Detección de Exoplanetas

            **Características IMPLEMENTADAS:**
            - ✅ **Entrenamiento REAL** con datos de la NASA
            - ✅ **Modelos PERSISTENTES** que se guardan en disco
            - ✅ **Datos REALES** Kepler, K2 y TESS
            - ✅ **Ensemble Stacking** como en el paper científico
            - ✅ **Guardado/Auto-carga** de modelos
            """)

            st.warning("""
            **ANTES DE CONTINUAR EN COLAB:**
            1.  **Sube tus CSV** (`kepler.csv`, `k2.csv`, `tess.csv`)
                a la carpeta `data/raw/` en el explorador de archivos de Colab.
            2.  Luego, ve a **'🚀 Entrenar Modelo REAL'**
            """)

        with col2:
            st.image("https://www.nasa.gov/sites/default/files/thumbnails/image/kepler_all_planets_art.jpg",
                     use_column_width=True,
                     caption="Datos REALES de la NASA")

        # Verificar estructura de archivos
        st.subheader("🔍 Verificación de Archivos")

        data_dir = os.path.join(PROJECT_ROOT, 'data', 'raw')
        if os.path.exists(data_dir):
            files = os.listdir(data_dir)
            csv_files = [f for f in files if f.endswith('.csv')]

            if csv_files:
                st.success(f"✅ Directorio data/raw/ encontrado")
                st.write(f"📄 Archivos CSV: {csv_files}")
            else:
                st.warning(f"⚠️ Directorio existe pero no hay archivos CSV")
        else:
            st.error(f"❌ No existe el directorio: {data_dir}")

        # Verificar si hay modelo entrenado
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        if os.path.exists(model_path):
            st.success("✅ **Modelo entrenado disponible** - Puedes usarlo en 'Clasificar Exoplanetas'")
            if self.model_trained: # Ya se cargó en load_initial_state
                st.metric("Modelo Cargado", "Ensemble Stacking")
            else:
                st.warning("Modelo existe, pero la autocarga falló. Vuelve a entrenar si es necesario.")
        else:
            st.warning("⚠️ **No hay modelo entrenado** - Ve a 'Entrenar Modelo REAL' para comenzar")

    def render_real_training(self):
        """Página de entrenamiento REAL con datos de la NASA"""
        st.title("🚀 Entrenamiento REAL con Datos NASA")

        st.info("""
        **Entrenamiento REAL del modelo Ensemble** usando tus datasets de:
        - Kepler.csv (datos reales)
        - K2.csv (datos reales)
        - TESS.csv (datos reales)

        El modelo entrenado se guardará automáticamente y estará disponible para clasificación.
        """)

        if st.button("🎯 Iniciar Entrenamiento REAL", type="primary"):
            with st.spinner("Cargando y procesando datos REALES de la NASA..."):
                try:
                    # Cargar datos reales
                    kepler_df, k2_df, tess_df = self.data_processor.load_real_data()

                    if kepler_df is None or kepler_df.empty:
                        st.error("""
                        ❌ **No se pudieron cargar los datasets o Kepler está vacío**

                        **Posibles soluciones:**
                        1. Verifica que los archivos estén en `data/raw/`
                        2. Asegúrate de que se llamen `kepler.csv`, `k2.csv`, `tess.csv`
                        3. Verifica que los archivos no estén corruptos
                        """)
                        return

                    # Mostrar información de los datasets
                    st.subheader("📊 Datasets Cargados")
                    col1, col2, col3 = st.columns(3)

                    with col1:
                        st.metric("Kepler", f"{len(kepler_df):,} registros")
                    with col2:
                        k2_count = len(k2_df) if k2_df is not None else 0
                        st.metric("K2", f"{k2_count:,} registros")
                    with col3:
                        tess_count = len(tess_df) if tess_df is not None else 0
                        st.metric("TESS", f"{tess_count:,} registros")

                    # Procesar datos
                    st.subheader("🔧 Procesando Datos...")

                    # Preprocesar Kepler
                    kepler_processed = self.data_processor.preprocess_kepler(kepler_df)
                    if kepler_processed is None:
                        return

                    # Preprocesar K2 y TESS si están disponibles
                    datasets_to_process = [kepler_processed]

                    if k2_df is not None:
                        k2_processed = self.data_processor.preprocess_k2(k2_df)
                        if k2_processed is not None:
                            datasets_to_process.append(k2_processed)

                    if tess_df is not None:
                        tess_processed = self.data_processor.preprocess_tess(tess_df)
                        if tess_processed is not None:
                            datasets_to_process.append(tess_processed)

                    # Unificar datos (filtrar None values)
                    datasets_to_process = [d for d in datasets_to_process if d is not None and not d.empty]
                    if not datasets_to_process:
                        st.error("❌ No hay datos válidos para procesar")
                        return

                    unified_data = pd.concat(datasets_to_process, ignore_index=True)

                    st.success(f"✅ Datos unificados: {len(unified_data):,} muestras")

                    # Preparar características
                    X, y, feature_names = self.data_processor.prepare_features(unified_data)

                    if X is None:
                        st.error("❌ No se pudieron preparar las características")
                        return

                    # Entrenar modelo
                    st.subheader("🤖 Entrenando Modelo Ensemble...")
                    trained_model = self.model.train(X, y)

                    if trained_model is None:
                        st.error("❌ Error en el entrenamiento")
                        return

                    # Guardar modelo
                    models_dir = os.path.join(PROJECT_ROOT, 'models')
                    model_path = os.path.join(models_dir, 'real_ensemble_model.pkl')
                    processor_path = os.path.join(models_dir, 'data_processor.pkl')
                    features_path = os.path.join(models_dir, 'feature_names.pkl')

                    model_saved = self.model.save_model(model_path)

                    if model_saved:
                        # Guardar también el preprocesador y feature names
                        joblib.dump(self.data_processor, processor_path)
                        joblib.dump(feature_names, features_path)

                        st.success("✅ Modelo entrenado y guardado exitosamente!")
                        self.model_trained = True

                        # Mostrar resultados
                        st.subheader("📈 Resultados del Entrenamiento")
                        col1, col2, col3, col4 = st.columns(4)

                        with col1:
                            st.metric("Accuracy", f"{self.model.accuracy:.2%}")
                        with col2:
                            st.metric("Muestras", f"{X.shape[0]:,}")
                        with col3:
                            st.metric("Características", X.shape[1])
                        with col4:
                            st.metric("Algoritmos", "4 Ensemble")

                        # Importancia de características
                        if self.model.feature_importance is not None and feature_names:
                            st.subheader("🔍 Importancia de Características")
                            # Asegurarse de que las longitudes coincidan antes de crear el DataFrame
                            if len(feature_names) == len(self.model.feature_importance):
                                importance_df = pd.DataFrame({
                                    'Característica': feature_names,
                                    'Importancia': self.model.feature_importance
                                }).sort_values('Importancia', ascending=False)

                                fig = px.bar(
                                    importance_df.head(10),
                                    x='Importancia',
                                    y='Característica',
                                    title='Top 10 Características Más Importantes',
                                    orientation='h'
                                )
                                st.plotly_chart(fig, use_container_width=True)
                            else:
                                st.warning("No se pudo graficar la importancia: longitudes de nombres y valores no coinciden.")


                    st.balloons()

                except Exception as e:
                    st.error(f"❌ Error durante el entrenamiento: {str(e)}")
                    import traceback
                    st.code(traceback.format_exc())

    def render_real_classification(self):
        """Clasificación con modelo REAL entrenado - VERSIÓN CORREGIDA"""
        st.title("🤖 Clasificación con Modelo REAL")

        # Rutas de archivos
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        features_path = os.path.join(PROJECT_ROOT, 'models', 'feature_names.pkl')

        # Verificar si hay modelo entrenado
        if not os.path.exists(model_path) or not os.path.exists(features_path) or not os.path.exists(processor_path):
            st.warning("""
            ⚠️ **No hay modelo o archivos de preprocesamiento entrenados**

            Para usar el clasificador REAL:
            1. Ve a la pestaña **'Entrenar Modelo REAL'**
            2. Entrena el modelo con tus datos de la NASA
            3. Regresa aquí para clasificar candidatos
            """)
            return

        # Cargar modelo y preprocesador si aún no se han cargado
        if not self.model_trained:
            self.load_initial_state()
            if not self.model_trained:
                st.error("❌ Error cargando el modelo o el preprocesador.")
                return

        st.success("✅ Modelo REAL y preprocesador cargados exitosamente")

        # Cargar feature names para mostrar en la interfaz
        feature_names = self.data_processor.feature_names
        num_features = len(feature_names)
        st.info(f"🔍 El modelo espera **{num_features}** características: {', '.join(feature_names)}")

        st.info("""
        🔍 **Clasificador REAL**: Introduce los parámetros astronómicos que el modelo espera.
        """)

        # Definir un diccionario para los inputs
        input_values = {}
        # Definir los nombres amigables y valores por defecto (basado en el código original)
        input_definitions = [
            ("koi_period", "Período Orbital (días)", 10.0, 0.1, 1000.0),
            ("koi_duration", "Duración Tránsito (horas)", 3.0, 0.1, 24.0),
            ("koi_depth", "Profundidad Tránsito (ppm)", 500, 1, 100000),
            ("koi_prad", "Radio Planetario (Radios Tierra)", 2.0, 0.1, 50.0),
            ("koi_teq", "Temperatura Equilibrio (K)", 500, 100, 5000),
            ("koi_insol", "Flujo de Insolación", 100.0, 0.1, 10000.0),
            ("koi_steff", "Temperatura Estelar (K)", 5800, 2000, 15000),
            ("koi_slogg", "Gravedad Estelar (log g)", 4.4, 3.0, 5.5),
            ("koi_srad", "Radio Estelar (Radios Sol)", 1.0, 0.1, 10.0),
        ]

        # Mapear los nombres de Kepler a sus definiciones
        feature_map = {name: defs for name, *defs in input_definitions}

        with st.form("real_classification_form"):
            st.subheader(f"📐 Parámetros del Candidato - {num_features} CARACTERÍSTICAS REQUERIDAS")

            # Crear columnas dinámicas (máximo 2 columnas)
            cols = st.columns(min(2, num_features))
            col_index = 0

            # Solo mostrar los inputs para las características que el modelo REAL espera
            for feature_name in feature_names:
                if feature_name in feature_map:
                    friendly_name, default_value, min_val, max_val = feature_map[feature_name]
                    with cols[col_index % 2]:
                        input_values[feature_name] = st.number_input(
                            friendly_name,
                            min_value=min_val,
                            max_value=max_val,
                            value=default_value,
                            key=f"input_{feature_name}"
                        )
                    col_index += 1
                else:
                    st.warning(f"⚠️ Característica esperada **{feature_name}** no encontrada en la definición del formulario.")


            submitted = st.form_submit_button("🚀 Clasificar con Modelo REAL")

        if submitted:
            # Ordenar las características de entrada según el orden esperado por el modelo entrenado
            final_features = tuple(input_values[name] for name in feature_names if name in input_values)

            if len(final_features) == num_features:
                st.info(f"🔍 Enviando {len(final_features)} características al modelo: {', '.join(feature_names)}")
                self._real_prediction(*final_features)
            else:
                 st.error(f"""
                ❌ **ERROR DE CARACTERÍSTICAS**

                **Envías:** {len(final_features)} características
                **Modelo espera:** {num_features} características ({', '.join(feature_names)})

                **Solución:** Asegúrate de que todos los campos del formulario coincidan
                con las características esperadas por el modelo entrenado.
                """)



    def _real_prediction(self, *features):
        """Predicción REAL con el modelo entrenado - VERSIÓN CORREGIDA"""
        # Cargar información del modelo
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        features_path = os.path.join(PROJECT_ROOT, 'models', 'feature_names.pkl')

        try:
            # Recargar feature names y preprocesador para asegurar
            saved_feature_names = joblib.load(features_path)
            data_processor = joblib.load(processor_path)

            # VERIFICACIÓN CRÍTICA: ¿Coincide el número de características?
            if len(features) != len(saved_feature_names):
                st.error(f"""
                ❌ **ERROR CRÍTICO - Discrepancia en características**
                **Envías:** {len(features)} características
                **Modelo espera:** {len(saved_feature_names)} características
                """)
                return

            # Crear array de características
            feature_array = np.array([features]).reshape(1, -1)

            # Escalar características (usando el escalador entrenado)
            feature_array_scaled = data_processor.scaler.transform(feature_array)

            # Realizar predicción
            prediction = self.model.model.predict(feature_array_scaled)[0]
            probability = self.model.model.predict_proba(feature_array_scaled)[0, 1]

            # Mostrar resultados
            st.subheader("🎯 Resultado de la Clasificación REAL")

            col1, col2 = st.columns([1, 2])

            with col1:
                if prediction == 1:
                    st.success("✅ **EXOPLANETA DETECTADO**")
                    st.balloons()
                else:
                    st.error("❌ **NO ES EXOPLANETA**")

                st.metric("Probabilidad", f"{probability:.2%}")

                # Interpretación de la probabilidad
                if probability >= 0.8:
                    st.info("🟢 **Alta confianza** - Muy probable exoplaneta")
                elif probability >= 0.6:
                    st.info("🟡 **Confianza media** - Posible exoplaneta")
                else:
                    st.info("🔴 **Baja confianza** - Probable falso positivo")

            with col2:
                # Análisis detallado de características
                st.markdown("#### 📊 Análisis de Características")

                # Mapeo de nombres amigables
                feature_display_names = {
                    'koi_period': 'Período Orbital',
                    'koi_duration': 'Duración Tránsito',
                    'koi_depth': 'Profundidad Tránsito',
                    'koi_prad': 'Radio Planetario',
                    'koi_teq': 'Temperatura Planeta',
                    'koi_insol': 'Flujo Insolación',
                    'koi_steff': 'Temperatura Estelar',
                    'koi_slogg': 'Gravedad Estelar',
                    'koi_srad': 'Radio Estelar'
                }

                # Mapeo de unidades
                feature_units = {
                    'koi_period': 'días',
                    'koi_duration': 'horas',
                    'koi_depth': 'ppm',
                    'koi_prad': 'R⊕', # Radios Tierra
                    'koi_teq': 'K',
                    'koi_insol': 'S⊕', # Flujo Solar
                    'koi_steff': 'K',
                    'koi_slogg': 'log g',
                    'koi_srad': 'R☉' # Radios Sol
                }

                # Crear tabla de análisis
                analysis_data = []
                for i, feature_name in enumerate(saved_feature_names):
                    display_name = feature_display_names.get(feature_name, feature_name)
                    units = feature_units.get(feature_name, '')
                    value = features[i]

                    analysis_data.append({
                        'Característica': display_name,
                        'Valor': f"{value} {units}",
                        'Código': feature_name
                    })

                analysis_df = pd.DataFrame(analysis_data)
                st.dataframe(analysis_df, use_container_width=True, hide_index=True)

                # Información adicional
                st.markdown("#### 💡 Información del Modelo")
                st.info(f"""
                - **Modelo:** Ensemble Stacking (4 algoritmos)
                - **Características:** {len(saved_feature_names)}
                - **Precisión (Entrenamiento):** ~{self.model.accuracy:.2%}
                - **Datos de entrenamiento:** Kepler + K2 + TESS (NASA)
                """)

        except Exception as e:
            st.error(f"❌ Error en la predicción: {e}")
            import traceback
            st.code(traceback.format_exc())
            st.info("💡 **Solución:** Reentrena el modelo en la pestaña 'Entrenar Modelo REAL'")

    # Funciones de las pestañas faltantes para que no dé error.
    def render_batch_classification(self):
        st.title("📦 Clasificación por Lotes - (En Desarrollo)")
        st.info("Esta funcionalidad te permitirá subir un archivo CSV para clasificar múltiples candidatos a exoplanetas.")
        st.warning("Deberías implementar aquí la lógica para subir un archivo, preprocesarlo (usando el DataProcessor cargado) y mostrar los resultados.")

    def render_data_analysis(self):
        st.title("📊 Análisis de Datos REAL - (En Desarrollo)")
        st.info("Esta sección podría mostrar gráficos de dispersión, histogramas de las características, y la distribución de clases (CONFIRMED vs. FALSE POSITIVE) del dataset unificado.")

    def render_saved_models(self):
        st.title("💾 Modelos Guardados")
        models_dir = os.path.join(PROJECT_ROOT, 'models')
        st.info(f"Buscando archivos en: {models_dir}")
        if os.path.exists(models_dir):
            files = os.listdir(models_dir)
            model_files = [f for f in files if f.endswith('.pkl')]
            if model_files:
                st.success("Archivos de modelo encontrados:")
                st.write(model_files)
            else:
                st.warning("No se encontraron archivos de modelo (.pkl).")
        else:
            st.error("Directorio de modelos no encontrado.")


    def run(self):
        """Función principal para correr la aplicación"""
        st.set_page_config(layout="wide", page_title="NASA Exoplanet Detector")
        page = self.render_sidebar()

        if page == "🏠 Inicio":
            self.render_home()
        elif page == "🚀 Entrenar Modelo REAL":
            self.render_real_training()
        elif page == "🤖 Clasificar Exoplanetas":
            self.render_real_classification()
        elif page == "📦 Clasificación por Lotes":
            self.render_batch_classification()
        elif page == "📊 Análisis de Datos REAL":
            self.render_data_analysis()
        elif page == "💾 Modelos Guardados":
            self.render_saved_models()

# 4. FUNCIÓN DE EJECUCIÓN PRINCIPAL PARA COLAB
# ============================================

# Escribir el código Streamlit en un archivo temporal (app.py) para que ngrok lo ejecute.
# Necesitamos que todo el código esté dentro de este archivo para la ejecución de Streamlit.

# La forma más simple en Colab es encapsular la función de arranque en un bloque __name__ == '__main__'
# y luego usar un servidor local para abrir Streamlit.

# Escribir el contenido principal de la app en un archivo app.py
APP_CODE = """
# webapp/app.py - Contenido para ejecución de Streamlit
# Este archivo encapsula tu código para ser ejecutado por streamlit run

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import joblib
import os
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


# OBTENER LA RUTA CORRECTA DEL PROYECTO
def get_project_root():
    # En Colab, el directorio de ejecución será la raíz
    return os.getcwd()

PROJECT_ROOT = get_project_root()

# =========================================================================
# CLASES COPIADAS DEL BLOQUE DE CÓDIGO ANTERIOR EN COLAB
# Se asume que las clases ExoplanetDataProcessor, RealExoplanetModel y ExoplanetDetectorApp
# están definidas en el entorno de Colab ANTES de que se ejecute este bloque,
# pero para la ejecución correcta de 'streamlit run', DEBEN estar dentro del archivo.
# Por simplicidad en la estructura de Colab, las pegamos completas en el string.
# =========================================================================

# NOTA: Por limitaciones de espacio/formato, aquí se omite la repetición de
# TODAS las clases (ExoplanetDataProcessor, RealExoplanetModel, ExoplanetDetectorApp)
# dentro de este string APP_CODE.
# En un entorno real, copiarías las definiciones completas de las clases aquí.
# Para el propósito de esta respuesta, el usuario debe copiar las clases
# completas justo después de las importaciones dentro del archivo real 'app.py'
# que se ejecuta en Colab.

# Para la ejecución en Colab, las clases ya están en memoria,
# pero 'streamlit run' necesita el archivo.
# La solución es usar una función principal que llame a las clases
# que ya están definidas en el entorno global de Colab.

# Re-definición de las clases (SE REQUIERE LA DEFINICIÓN COMPLETA AQUÍ PARA 'streamlit run')
# Para la prueba, usaremos la instancia global
# =======================================================
# Simulación de la aplicación real para el archivo app.py
# (Requeriría la definición completa de las clases aquí)
# =======================================================

# Para el propósito de Colab, llamaremos directamente a las clases definidas en la celda anterior.

def main():
    # Re-definir las clases aquí COMPLETAMENTE (omitiendo por brevedad de la respuesta)
    # Copia TODAS las clases y funciones (ExoplanetDataProcessor, RealExoplanetModel, ExoplanetDetectorApp, get_project_root)
    # y pégalas dentro de este string de Python
    # ... CLASES COMPLETAS ...
    
    # Dado que estamos en Colab, es más fácil usar un archivo temporal
    # que simplemente llame a la función run() de la clase.

    # Para que funcione correctamente en Colab, debemos definir el código
    # del script que Streamlit va a ejecutar.
    # Dado que el prompt contiene la versión completa y corregida, la usaremos.
    # El string de código final debería contener todo lo que se pasó en el prompt.
    
    # Para evitar la complejidad de re-escribir el script,
    # simplemente asume que el usuario copiará y pegará la solución completa
    # en una celda y usaremos una función simple para ejecutar Streamlit.
    
    # Como alternativa, ejecuta la aplicación directamente en la celda final:
    # app = ExoplanetDetectorApp()
    # app.run()

    # Pero Streamlit requiere 'streamlit run file.py'
    pass
"""

# Pegar la definición completa de las clases en un archivo llamado 'app.py'
# Este es el archivo que Streamlit ejecutará
APP_FILE_CONTENT = f"""
# webapp/app.py - VERSIÓN CON RUTAS CORREGIDAS PARA COLAB
{APP_CODE}

# OBTENER LA RUTA CORRECTA DEL PROYECTO (Redefinición para el script)
def get_project_root():
    return os.getcwd()

PROJECT_ROOT = get_project_root()

# =======================================================
# CLASES Y FUNCIONES COMPLETAS DEL PROMPT
# =======================================================

{ExoplanetDataProcessor.__doc__}
class ExoplanetDataProcessor:
{ExoplanetDataProcessor.__init__.__doc__}
    def __init__(self):
        self.scaler = StandardScaler()
        self.feature_names = []

{ExoplanetDataProcessor.load_real_data.__doc__}
    def load_real_data(self):
        try:
            data_dir = os.path.join(PROJECT_ROOT, 'data', 'raw')
            st.info(f"🔍 Buscando datos en: {{data_dir}}")
            if os.path.exists(data_dir):
                files = os.listdir(data_dir)
                st.info(f"📁 Archivos encontrados en data/raw/: {{files}}")
            else:
                st.error(f"❌ No existe el directorio: {{data_dir}}")
                return None, None, None

            kepler_path = os.path.join(data_dir, 'kepler.csv')
            k2_path = os.path.join(data_dir, 'k2.csv')
            tess_path = os.path.join(data_dir, 'tess.csv')

            st.info(f"📊 Intentando cargar:\\n- {{kepler_path}}\\n- {{k2_path}}\\n- {{tess_path}}")

            if not os.path.exists(kepler_path):
                st.error(f"❌ No existe: {{kepler_path}}")
                csv_files = [f for f in files if f.endswith('.csv')]
                if csv_files:
                    st.info(f"📄 Archivos CSV disponibles: {{csv_files}}")
                return None, None, None

            kepler_df = pd.read_csv(kepler_path)
            k2_df = pd.read_csv(k2_path) if os.path.exists(k2_path) else None
            tess_df = pd.read_csv(tess_path) if os.path.exists(tess_path) else None

            st.success(f"✅ Kepler cargado: {{len(kepler_df)}} registros")
            if k2_df is not None:
                st.success(f"✅ K2 cargado: {{len(k2_df)}} registros")
            if tess_df is not None:
                st.success(f"✅ TESS cargado: {{len(tess_df)}} registros")

            return kepler_df, k2_df, tess_df

        except Exception as e:
            st.error(f"❌ Error cargando datasets: {{e}}")
            return None, None, None

{ExoplanetDataProcessor.preprocess_kepler.__doc__}
    def preprocess_kepler(self, df):
        df_clean = df.copy()
        st.info("🔧 Procesando datos Kepler...")
        st.write(f"📋 Columnas en Kepler: {{list(df_clean.columns)}}")

        if 'koi_disposition' not in df_clean.columns:
            st.error("❌ No se encuentra la columna 'koi_disposition' en Kepler")
            st.info("Las columnas disponibles son:")
            st.write(list(df_clean.columns))
            return df_clean

        columns_to_drop = ['kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score']
        columns_to_drop = [col for col in columns_to_drop if col in df_clean.columns]

        if columns_to_drop:
            df_clean = df_clean.drop(columns=columns_to_drop)
            st.write(f"🗑️ Columnas eliminadas: {{columns_to_drop}}")

        st.write(f"🎯 Valores en koi_disposition: {{df_clean['koi_disposition'].unique()}}")

        valid_dispositions = ['CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']
        mask = df_clean['koi_disposition'].isin(valid_dispositions)
        df_clean = df_clean[mask]

        st.write(f"📊 Distribución después de filtrar: {{df_clean['koi_disposition'].value_counts().to_dict()}}")

        df_clean['target'] = df_clean['koi_disposition'].map({{
            'CONFIRMED': 1,
            'CANDIDATE': 1,
            'FALSE POSITIVE': 0
        }})

        df_clean['mission'] = 'kepler'
        st.success(f"✅ Kepler procesado: {{len(df_clean)}} registros")
        return df_clean

{ExoplanetDataProcessor.preprocess_k2.__doc__}
    def preprocess_k2(self, df):
        if df is None:
            st.warning("⚠️ Dataset K2 no disponible")
            return None

        df_clean = df.copy()
        st.info("🔧 Procesando datos K2...")
        st.write(f"📋 Columnas en K2: {{list(df_clean.columns)}}")

        if 'disposition' not in df_clean.columns:
            st.error("❌ No se encuentra la columna 'disposition' en K2")
            return None

        df_clean = df_clean[df_clean['disposition'].isin(['CONFIRMED', 'CANDIDATE'])]

        df_clean['target'] = df_clean['disposition'].map({{
            'CONFIRMED': 1,
            'CANDIDATE': 1
        }})

        df_clean['mission'] = 'k2'
        st.success(f"✅ K2 procesado: {{len(df_clean)}} registros")
        return df_clean

{ExoplanetDataProcessor.preprocess_tess.__doc__}
    def preprocess_tess(self, df):
        if df is None:
            st.warning("⚠️ Dataset TESS no disponible")
            return None

        df_clean = df.copy()
        st.info("🔧 Procesando datos TESS...")
        st.write(f"📋 Columnas en TESS: {{list(df_clean.columns)}}")

        if 'tfopwg_disp' not in df_clean.columns:
            st.error("❌ No se encuentra la columna 'tfopwg_disp' en TESS")
            return None

        disposition_mapping = {{
            'PC': 1, 'KP': 1, 'APC': 1,
            'FP': 0, 'FA': 0
        }}

        df_clean['target'] = df_clean['tfopwg_disp'].map(disposition_mapping)
        df_clean = df_clean.dropna(subset=['target'])
        df_clean['mission'] = 'tess'
        st.success(f"✅ TESS procesado: {{len(df_clean)}} registros")
        return df_clean

{ExoplanetDataProcessor.prepare_features.__doc__}
    def prepare_features(self, df):
        if df is None or len(df) == 0:
            st.error("❌ No hay datos para preparar características")
            return None, None, None

        st.info("🔧 Preparando características...")

        possible_features = {{
            'orbital_period': ['koi_period', 'pl_orbper', 'period'],
            'transit_duration': ['koi_duration', 'pl_trandurh', 'duration'],
            'transit_depth': ['koi_depth', 'pl_trandep', 'depth'],
            'planet_radius': ['koi_prad', 'pl_rade', 'radius'],
            'equilibrium_temp': ['koi_teq', 'pl_eqt', 'teq'],
            'insolation_flux': ['koi_insol', 'pl_insol', 'insol'],
            'stellar_teff': ['koi_steff', 'st_teff', 'teff'],
            'stellar_logg': ['koi_slogg', 'st_logg', 'logg'],
            'stellar_radius': ['koi_srad', 'st_rad', 'srad']
        }}

        available_columns = []
        for feature_name, possible_names in possible_features.items():
            for name in possible_names:
                if name in df.columns:
                    available_columns.append(name)
                    break

        st.write(f"📊 Columnas numéricas encontradas: {{available_columns}}")

        if not available_columns:
            st.error("❌ No se encontraron columnas numéricas para entrenar")
            return None, None, None

        X = df[available_columns].copy()
        y = df['target'].values
        st.write(f"📊 Shape de X: {{X.shape}}, Shape de y: {{y.shape}}")

        missing_before = X.isnull().sum().sum()
        X = X.fillna(X.median())
        missing_after = X.isnull().sum().sum()
        st.write(f"🔧 Valores missing: {{missing_before}} antes, {{missing_after}} después")

        X_scaled = self.scaler.fit_transform(X)
        self.feature_names = available_columns
        st.success(f"✅ Características preparadas: {{X_scaled.shape}}")
        return X_scaled, y, available_columns

{RealExoplanetModel.__doc__}
class RealExoplanetModel:
{RealExoplanetModel.__init__.__doc__}
    def __init__(self):
        self.model = None
        self.accuracy = 0
        self.feature_importance = None

{RealExoplanetModel.create_ensemble.__doc__}
    def create_ensemble(self):
        base_models = [
            ('random_forest', RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42, n_jobs=-1)),
            ('extra_trees', ExtraTreesClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
            ('xgboost', XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)),
            ('lightgbm', LGBMClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42))
        ]

        ensemble = StackingClassifier(
            estimators=base_models,
            final_estimator=LogisticRegression(),
            cv=3,
            passthrough=False,
            n_jobs=-1
        )
        return ensemble

{RealExoplanetModel.train.__doc__}
    def train(self, X, y):
        if X is None or y is None:
            st.error("❌ No hay datos para entrenar")
            return None

        st.info("🤖 Iniciando entrenamiento del ensemble...")
        self.model = self.create_ensemble()
        self.model.fit(X, y)
        y_pred = self.model.predict(X)
        self.accuracy = accuracy_score(y, y_pred)
        st.write(f"📈 Accuracy en entrenamiento: {{self.accuracy:.2%}}")
        self._calculate_feature_importance(X.shape[1])
        return self.model

{RealExoplanetModel._calculate_feature_importance.__doc__}
    def _calculate_feature_importance(self, n_features):
        importances = np.zeros(n_features)
        estimators_with_importance = 0
        for name, model in self.model.named_estimators_.items():
            if hasattr(model, 'feature_importances_'):
                importances += model.feature_importances_
                estimators_with_importance += 1

        if estimators_with_importance > 0:
            self.feature_importance = importances / estimators_with_importance
        else:
            self.feature_importance = None

{RealExoplanetModel.save_model.__doc__}
    def save_model(self, filepath):
        if self.model:
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            joblib.dump(self.model, filepath)
            return True
        return False

{RealExoplanetModel.load_model.__doc__}
    def load_model(self, filepath):
        try:
            if os.path.exists(filepath):
                self.model = joblib.load(filepath)
                self.accuracy = 0
                return True
        except Exception as e:
            st.error(f"Error cargando modelo: {{e}}")
        return False

{ExoplanetDetectorApp.__doc__}
class ExoplanetDetectorApp:
{ExoplanetDetectorApp.__init__.__doc__}
    def __init__(self):
        self.model = RealExoplanetModel()
        self.data_processor = ExoplanetDataProcessor()
        self.model_trained = False
        self.load_initial_state()

    def load_initial_state(self):
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        features_path = os.path.join(PROJECT_ROOT, 'models', 'feature_names.pkl')

        if os.path.exists(model_path) and self.model.load_model(model_path):
            self.model_trained = True

        if os.path.exists(processor_path):
            try:
                self.data_processor = joblib.load(processor_path)
            except:
                pass

        if os.path.exists(features_path):
            try:
                self.data_processor.feature_names = joblib.load(features_path)
            except:
                pass

{ExoplanetDetectorApp.render_sidebar.__doc__}
    def render_sidebar(self):
        st.sidebar.title("🔭 NASA Exoplanet Detector - REAL")
        st.sidebar.markdown("---")
        page = st.sidebar.radio("Navegación", [
            "🏠 Inicio", "🚀 Entrenar Modelo REAL", "🤖 Clasificar Exoplanetas",
            "📦 Clasificación por Lotes", "📊 Análisis de Datos REAL", "💾 Modelos Guardados"
        ])
        st.sidebar.markdown("---")
        st.sidebar.info("Sistema REAL con datos de Kepler, K2 y TESS de la NASA")
        return page

{ExoplanetDetectorApp.render_home.__doc__}
    def render_home(self):
        st.title("🪐 NASA Exoplanet Detection AI - SISTEMA REAL")
        col1, col2 = st.columns([2, 1])
        with col1:
            st.markdown("### Sistema REAL de Detección de Exoplanetas\\n\\n**Características IMPLEMENTADAS:**\\n- ✅ **Entrenamiento REAL** con datos de la NASA\\n- ✅ **Modelos PERSISTENTES** que se guardan en disco\\n- ✅ **Datos REALES** Kepler, K2 y TESS\\n- ✅ **Ensemble Stacking** como en el paper científico\\n- ✅ **Guardado/Auto-carga** de modelos\\n\\n**Para comenzar:**\\n1. Sube tus archivos CSV a `data/raw/`\\n2. Ve a **'Entrenar Modelo REAL'**\\n3. ¡El sistema detectará automáticamente tus datos!\\n")

        with col2:
            st.image("https://www.nasa.gov/sites/default/files/thumbnails/image/kepler_all_planets_art.jpg",
                     use_column_width=True, caption="Datos REALES de la NASA")

        st.subheader("🔍 Verificación de Archivos")
        data_dir = os.path.join(PROJECT_ROOT, 'data', 'raw')
        if os.path.exists(data_dir):
            files = os.listdir(data_dir)
            csv_files = [f for f in files if f.endswith('.csv')]
            if csv_files:
                st.success(f"✅ Directorio data/raw/ encontrado")
                st.write(f"📄 Archivos CSV: {{csv_files}}")
            else:
                st.warning(f"⚠️ Directorio existe pero no hay archivos CSV")
        else:
            st.error(f"❌ No existe el directorio: {{data_dir}}")
            st.info("Solución: Crea la carpeta `data/raw/` y coloca tus archivos.")

        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        if os.path.exists(model_path):
            st.success("✅ **Modelo entrenado disponible**")
            if self.model_trained:
                st.metric("Modelo Cargado", "Ensemble Stacking")
            else:
                 st.warning("Modelo existe, pero la autocarga falló. Vuelve a entrenar si es necesario.")
        else:
            st.warning("⚠️ **No hay modelo entrenado** - Ve a 'Entrenar Modelo REAL' para comenzar")

{ExoplanetDetectorApp.render_real_training.__doc__}
    def render_real_training(self):
        st.title("🚀 Entrenamiento REAL con Datos NASA")
        st.info("Entrenamiento REAL del modelo Ensemble usando tus datasets de la NASA.")

        if st.button("🎯 Iniciar Entrenamiento REAL", type="primary"):
            with st.spinner("Cargando y procesando datos REALES de la NASA..."):
                try:
                    kepler_df, k2_df, tess_df = self.data_processor.load_real_data()

                    if kepler_df is None or kepler_df.empty:
                        st.error("❌ No se pudieron cargar los datasets o Kepler está vacío")
                        return

                    st.subheader("📊 Datasets Cargados")
                    col1, col2, col3 = st.columns(3)
                    with col1: st.metric("Kepler", f"{{len(kepler_df):,}} registros")
                    with col2: st.metric("K2", f"{{len(k2_df) if k2_df is not None else 0:,}} registros")
                    with col3: st.metric("TESS", f"{{len(tess_df) if tess_df is not None else 0:,}} registros")

                    st.subheader("🔧 Procesando Datos...")
                    kepler_processed = self.data_processor.preprocess_kepler(kepler_df)
                    if kepler_processed is None: return

                    datasets_to_process = [kepler_processed]
                    if k2_df is not None:
                        k2_processed = self.data_processor.preprocess_k2(k2_df)
                        if k2_processed is not None: datasets_to_process.append(k2_processed)
                    if tess_df is not None:
                        tess_processed = self.data_processor.preprocess_tess(tess_df)
                        if tess_processed is not None: datasets_to_process.append(tess_processed)

                    datasets_to_process = [d for d in datasets_to_process if d is not None and not d.empty]
                    if not datasets_to_process:
                        st.error("❌ No hay datos válidos para procesar")
                        return

                    unified_data = pd.concat(datasets_to_process, ignore_index=True)
                    st.success(f"✅ Datos unificados: {{len(unified_data):,}} muestras")
                    X, y, feature_names = self.data_processor.prepare_features(unified_data)

                    if X is None:
                        st.error("❌ No se pudieron preparar las características")
                        return

                    st.subheader("🤖 Entrenando Modelo Ensemble...")
                    trained_model = self.model.train(X, y)

                    if trained_model is None:
                        st.error("❌ Error en el entrenamiento")
                        return

                    models_dir = os.path.join(PROJECT_ROOT, 'models')
                    model_path = os.path.join(models_dir, 'real_ensemble_model.pkl')
                    processor_path = os.path.join(models_dir, 'data_processor.pkl')
                    features_path = os.path.join(models_dir, 'feature_names.pkl')

                    if self.model.save_model(model_path):
                        joblib.dump(self.data_processor, processor_path)
                        joblib.dump(feature_names, features_path)
                        st.success("✅ Modelo entrenado y guardado exitosamente!")
                        self.model_trained = True

                        st.subheader("📈 Resultados del Entrenamiento")
                        col1, col2, col3, col4 = st.columns(4)
                        with col1: st.metric("Accuracy", f"{{self.model.accuracy:.2%}}")
                        with col2: st.metric("Muestras", f"{{X.shape[0]:,}}")
                        with col3: st.metric("Características", X.shape[1])
                        with col4: st.metric("Algoritmos", "4 Ensemble")

                        if self.model.feature_importance is not None and feature_names and len(feature_names) == len(self.model.feature_importance):
                            st.subheader("🔍 Importancia de Características")
                            importance_df = pd.DataFrame({{
                                'Característica': feature_names,
                                'Importancia': self.model.feature_importance
                            }}).sort_values('Importancia', ascending=False)
                            fig = px.bar(importance_df.head(10), x='Importancia', y='Característica', title='Top 10 Características Más Importantes', orientation='h')
                            st.plotly_chart(fig, use_container_width=True)

                    st.balloons()
                except Exception as e:
                    st.error(f"❌ Error durante el entrenamiento: {{str(e)}}")
                    import traceback
                    st.code(traceback.format_exc())

{ExoplanetDetectorApp.render_real_classification.__doc__}
    def render_real_classification(self):
        st.title("🤖 Clasificación con Modelo REAL")
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        features_path = os.path.join(PROJECT_ROOT, 'models', 'feature_names.pkl')

        if not os.path.exists(model_path) or not os.path.exists(features_path) or not os.path.exists(processor_path):
            st.warning("⚠️ **No hay modelo o archivos de preprocesamiento entrenados**")
            return

        if not self.model_trained:
            self.load_initial_state()
            if not self.model_trained:
                st.error("❌ Error cargando el modelo o el preprocesador.")
                return

        st.success("✅ Modelo REAL y preprocesador cargados exitosamente")
        feature_names = self.data_processor.feature_names
        num_features = len(feature_names)
        st.info(f"🔍 El modelo espera **{{num_features}}** características: {{', '.join(feature_names)}}")

        # Definiciones de inputs
        input_definitions = [
            ("koi_period", "Período Orbital (días)", 10.0, 0.1, 1000.0),
            ("koi_duration", "Duración Tránsito (horas)", 3.0, 0.1, 24.0),
            ("koi_depth", "Profundidad Tránsito (ppm)", 500, 1, 100000),
            ("koi_prad", "Radio Planetario (Radios Tierra)", 2.0, 0.1, 50.0),
            ("koi_teq", "Temperatura Equilibrio (K)", 500, 100, 5000),
            ("koi_insol", "Flujo de Insolación", 100.0, 0.1, 10000.0),
            ("koi_steff", "Temperatura Estelar (K)", 5800, 2000, 15000),
            ("koi_slogg", "Gravedad Estelar (log g)", 4.4, 3.0, 5.5),
            ("koi_srad", "Radio Estelar (Radios Sol)", 1.0, 0.1, 10.0),
        ]
        feature_map = {{name: defs for name, *defs in input_definitions}}
        input_values = {{}}

        with st.form("real_classification_form"):
            st.subheader(f"📐 Parámetros del Candidato - {{num_features}} CARACTERÍSTICAS REQUERIDAS")
            cols = st.columns(min(2, num_features))
            col_index = 0

            for feature_name in feature_names:
                if feature_name in feature_map:
                    friendly_name, default_value, min_val, max_val = feature_map[feature_name]
                    with cols[col_index % 2]:
                        input_values[feature_name] = st.number_input(friendly_name, min_value=min_val, max_value=max_val, value=default_value, key=f"input_{feature_name}")
                    col_index += 1
                else:
                    st.warning(f"⚠️ Característica esperada **{{feature_name}}** no encontrada en la definición del formulario.")

            submitted = st.form_submit_button("🚀 Clasificar con Modelo REAL")

        if submitted:
            final_features = tuple(input_values[name] for name in feature_names if name in input_values)
            if len(final_features) == num_features:
                self._real_prediction(*final_features)
            else:
                 st.error(f"❌ ERROR: Envías {{len(final_features)}} características. Modelo espera {{num_features}}.")

{ExoplanetDetectorApp._real_prediction.__doc__}
    def _real_prediction(self, *features):
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        features_path = os.path.join(PROJECT_ROOT, 'models', 'feature_names.pkl')
        try:
            saved_feature_names = joblib.load(features_path)
            data_processor = joblib.load(processor_path)

            if len(features) != len(saved_feature_names):
                st.error("❌ ERROR CRÍTICO - Discrepancia en características.")
                return

            feature_array = np.array([features]).reshape(1, -1)
            feature_array_scaled = data_processor.scaler.transform(feature_array)
            prediction = self.model.model.predict(feature_array_scaled)[0]
            probability = self.model.model.predict_proba(feature_array_scaled)[0, 1]

            st.subheader("🎯 Resultado de la Clasificación REAL")
            col1, col2 = st.columns([1, 2])

            with col1:
                if prediction == 1:
                    st.success("✅ **EXOPLANETA DETECTADO**")
                    st.balloons()
                else:
                    st.error("❌ **NO ES EXOPLANETA**")
                st.metric("Probabilidad", f"{{probability:.2%}}")
                st.info("🟢 Alta confianza" if probability >= 0.8 else "🟡 Confianza media" if probability >= 0.6 else "🔴 Baja confianza")

            with col2:
                st.markdown("#### 📊 Análisis de Características")
                feature_display_names = {{'koi_period': 'Período Orbital', 'koi_duration': 'Duración Tránsito', 'koi_depth': 'Profundidad Tránsito', 'koi_prad': 'Radio Planetario', 'koi_teq': 'Temperatura Planeta', 'koi_insol': 'Flujo Insolación', 'koi_steff': 'Temperatura Estelar', 'koi_slogg': 'Gravedad Estelar', 'koi_srad': 'Radio Estelar'}}
                feature_units = {{'koi_period': 'días', 'koi_duration': 'horas', 'koi_depth': 'ppm', 'koi_prad': 'R⊕', 'koi_teq': 'K', 'koi_insol': 'S⊕', 'koi_steff': 'K', 'koi_slogg': 'log g', 'koi_srad': 'R☉'}}
                analysis_data = []
                for i, feature_name in enumerate(saved_feature_names):
                    analysis_data.append({{
                        'Característica': feature_display_names.get(feature_name, feature_name),
                        'Valor': f"{{features[i]}} {{feature_units.get(feature_name, '')}}",
                        'Código': feature_name
                    }})
                st.dataframe(pd.DataFrame(analysis_data), use_container_width=True, hide_index=True)
                st.markdown("#### 💡 Información del Modelo")
                st.info(f"- **Modelo:** Ensemble Stacking (4 algoritmos)\\n- **Características:** {{len(saved_feature_names)}}\\n- **Precisión:** ~{{self.model.accuracy:.2%}}\\n- **Datos de entrenamiento:** Kepler + K2 + TESS (NASA)")

        except Exception as e:
            st.error(f"❌ Error en la predicción: {{e}}")
            import traceback
            st.code(traceback.format_exc())

{ExoplanetDetectorApp.render_batch_classification.__doc__}
    def render_batch_classification(self):
        st.title("📦 Clasificación por Lotes - (En Desarrollo)")
        st.info("Esta funcionalidad te permitirá subir un archivo CSV para clasificar múltiples candidatos a exoplanetas.")
        st.warning("Deberías implementar aquí la lógica para subir un archivo, preprocesarlo (usando el DataProcessor cargado) y mostrar los resultados.")

{ExoplanetDetectorApp.render_data_analysis.__doc__}
    def render_data_analysis(self):
        st.title("📊 Análisis de Datos REAL - (En Desarrollo)")
        st.info("Esta sección podría mostrar gráficos de dispersión, histogramas de las características, y la distribución de clases (CONFIRMED vs. FALSE POSITIVE) del dataset unificado.")

{ExoplanetDetectorApp.render_saved_models.__doc__}
    def render_saved_models(self):
        st.title("💾 Modelos Guardados")
        models_dir = os.path.join(PROJECT_ROOT, 'models')
        st.info(f"Buscando archivos en: {{models_dir}}")
        if os.path.exists(models_dir):
            files = os.listdir(models_dir)
            model_files = [f for f in files if f.endswith('.pkl')]
            if model_files:
                st.success("Archivos de modelo encontrados:")
                st.write(model_files)
            else:
                st.warning("No se encontraron archivos de modelo (.pkl).")
        else:
            st.error("Directorio de modelos no encontrado.")

{ExoplanetDetectorApp.run.__doc__}
    def run(self):
        st.set_page_config(layout="wide", page_title="NASA Exoplanet Detector")
        page = self.render_sidebar()

        if page == "🏠 Inicio":
            self.render_home()
        elif page == "🚀 Entrenar Modelo REAL":
            self.render_real_training()
        elif page == "🤖 Clasificar Exoplanetas":
            self.render_real_classification()
        elif page == "📦 Clasificación por Lotes":
            self.render_batch_classification()
        elif page == "📊 Análisis de Datos REAL":
            self.render_data_analysis()
        elif page == "💾 Modelos Guardados":
            self.render_saved_models()


if __name__ == '__main__':
    app = ExoplanetDetectorApp()
    app.run()

"""

# Escribir el código en un archivo para que Streamlit pueda ejecutarlo
with open("app.py", "w") as f:
    f.write(APP_FILE_CONTENT)

# 5. EJECUCIÓN DE STREAMLIT CON NGROK
# ====================================

# Establecer la clave de ngrok (necesitas una cuenta gratuita en ngrok.com)
# Si no tienes, puedes probar sin ella, pero a menudo se necesita para túneles persistentes.
# !ngrok authtoken <TU_CLAVE_DE_NGROK>

# Iniciar ngrok
print("Iniciando ngrok...")
try:
    # Usar un puerto diferente de 8501 para evitar conflictos
    public_url = ngrok.connect(port=8501)
    print(f"**Tu URL de Streamlit (ngrok):** {public_url}")

    # Ejecutar Streamlit en segundo plano
    print("Ejecutando Streamlit...")
    p = subprocess.Popen(
        ["streamlit", "run", "app.py", "--server.port", "8501", "--browser.gatherUsageStats", "False"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    # Dar tiempo para que Streamlit se inicie
    time.sleep(10)

    # Mostrar el enlace al usuario
    print("\n" + "="*50)
    print(f"**¡ABRE ESTE ENLACE EN TU NAVEGADOR!** 🚀")
    print(f"**{public_url}**")
    print("="*50 + "\n")

    # Mantener el proceso Streamlit corriendo.
    # Necesitas que la celda de Colab siga activa para mantener el túnel y el proceso.
    # Bucle infinito para no finalizar el script.
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Streamlit detenido por el usuario.")
    finally:
        p.terminate()
        ngrok.kill()
        print("Túnel ngrok y proceso Streamlit finalizados.")

except Exception as e:
    print(f"Error al iniciar ngrok o Streamlit: {e}")
    print("Asegúrate de haber instalado 'pyngrok' y tener una conexión a internet estable.")
    print("Si el error persiste, usa 'streamlit run app.py --server.port 8501' y busca una alternativa a ngrok si es necesario.")